The main solving big ridge regression problems

In [1]:
workspace()

In [2]:
using Plots

In [3]:
using StatsBase
using JLD  
include("../src/BigRidge.jl")

plot_outputs_Plots (generic function with 1 method)

In [4]:
## Basic parameters
maxiter=2000;
max_time = 200;
max_epocs = 100;
printiters = true;
exacterror =true; 
tol = 10.0^(-5.0);
restol = 10.0^(-5.0);
skip_error_calculation = 25;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))

In [5]:
dataset = "a9a"   # news20.binary  rcv1_train.multiclass rcv1_train.binary mnist.scale  a9a  gisette_scale 
prob = defineProb(dataset);
showDetails(dataset)

../data/a9a.jld


LoadError: File ../data/details.jld cannot be found

In [6]:
norm(prob.A*prob.xsol - prob.b)

29961.767653199953

In [7]:
options = MyOptions(0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror,[]) 
OUTPUTS = [];

In [118]:
options.AUX = [0 2]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = floor((prob.n)^(1/2)); #
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 5]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 50; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 10]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 5; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.AUX = [0 50]; # don't use sign flipping and use a density of 10 per row
method_name = "rademacher";# ROS
options.sketchsize = 2; #floor((prob.n)^(1/2));
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

rademacher-217-2
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 39.90  |  22.08  | 17.8263 
 50  | 26.28  |  12.41  | 36.3336 
 75  | 19.18  |  8.85  | 53.4713 
100  | 14.14  |  7.15  | 71.2183 
125  | 11.92  |  6.00  | 90.7484 
150  | 9.80  |  4.97  | 107.9083 
175  | 8.00  |  4.26  | 123.8179 
200  | 7.18  |  3.79  | 139.9509 
225  | 6.28  |  3.46  | 155.9391 
250  | 5.59  |  3.24  | 172.2164 
275  | 4.97  |  2.88  | 188.0398 
293  | 4.61  |  2.51  | 200.2195 
226.352055 seconds (1.80 M allocations: 67.752 GB, 2.80% gc time)
rademacher-50-5
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 59.03  |  38.60  | 6.8816 
 50  | 47.16  |  22.12  | 14.3426 
 75  | 39.84  |  16.04  | 21.5649 
100  | 33.98  |  12.78  | 28.7581 
125  | 30.43  |  11.89  | 35.6075 
150  | 27.35  |  10.35  | 42.9313 
175  | 24.78  |  9.42  | 49.5897 
200  | 22.77  |  8.85  | 56.9936 
225  | 21.03  |  8.13  | 63.8718 
250  | 19.42  |  7.65  | 71.

In [98]:
# Testing Hadamard Sketch
# method_name = "Hadamard";# ROS
# options.sketchsize =10; # Int(floor(log2(prob.n)));
# options.skip_error_calculation =25;
# @time  output= solveRidge(prob, method_name, options);
# OUTPUTS = [OUTPUTS ; output];

In [99]:
# Testing countmin 
options.sketchsize =10;#floor((prob.n)^(1/2));
options.skip_error_calculation =25;
method_name = "countmin";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

In [119]:
# Testing CG 
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CG
-------------------
It   | Error% | Residual |  Time   
-------------------
 25  | 3.39  |  4.19  | 24.3342 
 50  | 1.12  |  1.00  | 49.5141 
 75  | 0.60  |  0.55  | 73.9979 
100  | 0.39  |  0.38  | 99.0870 
125  | 0.98  |  0.31  | 125.1789 
150  | 0.61  |  0.29  | 150.9208 
175  | 0.98  |  0.18  | 177.3980 
200  | 0.14  |  0.16  | 199.8178 
201  | 0.14  |  0.19  | 200.5961 
222.282549 seconds (23.17 k allocations: 599.786 MB, 0.11% gc time)


In [120]:
# Testing CD 
options.maxiter=5000;
options.skip_error_calculation = 250;
method_name = "CD";# gradient descent
options.sketchsize = 10;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
method_name = "CD";# gradient descent
options.sketchsize = 50;
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

CD-10
-------------------
It   | Error% | Residual |  Time   
-------------------
 64  | 89.79  |  84.95  | 0.5997 
100  | 83.05  |  76.49  | 1.1823 
150  | 76.15  |  65.61  | 1.5997 
200  | 72.23  |  60.91  | 2.1854 
250  | 67.65  |  54.88  | 2.6127 
300  | 63.25  |  48.18  | 3.0807 
350  | 59.64  |  43.07  | 3.4940 
400  | 56.24  |  40.49  | 3.9414 
450  | 54.03  |  37.71  | 4.6443 
500  | 52.80  |  35.67  | 5.2829 
550  | 50.13  |  32.75  | 5.9706 
600  | 48.65  |  31.31  | 6.4587 
650  | 47.48  |  30.03  | 7.0483 
700  | 45.62  |  28.43  | 7.6835 
750  | 44.62  |  27.78  | 8.3260 
800  | 43.79  |  26.16  | 8.7347 
850  | 41.78  |  23.79  | 9.2381 
900  | 40.67  |  22.79  | 9.9001 
950  | 38.24  |  21.45  | 10.5192 
1000  | 37.96  |  20.89  | 11.0188 
1050  | 36.13  |  19.75  | 11.8788 
1100  | 36.39  |  19.26  | 12.3454 
1150  | 35.18  |  18.50  | 13.0246 
1200  | 34.10  |  18.14  | 13.4896 
1250  | 33.27  |  17.10  | 13.9273 
1300  | 32.63  |  16.77  | 14.5026 
1350  | 31.96  |  1

In [121]:
# OUTPUTS =[];
# include("../src/call_all_methods.jl")

In [122]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename)-OUTPUTS.jld", "OUTPUTS", OUTPUTS)

In [123]:
#using GR
using Plots
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

rcv1_train.multiclass


In [344]:
# Loading saved problem data
probname = "gisette_scale"#"gisette_scale";  
test = load("../data/$(probname)-OUTPUTS.jld")
OUTPUTS = test["OUTPUTS"];

In [125]:
OUTPUTS = []
options.tol = 10.0^(-2.0);
options.restol = 10.0^(-2.0);
options.maxiter=9000000;
options.max_time = 300;
options.skip_error_calculation = 100;
options.AUX = [0 10] # don't use sign flipping and use a density of 5 per row
method_name = "rademacher";# ROS
options.sketchsize = 100; #
@time   output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.skip_error_calculation = 10;
method_name = "CG";# gradient descent
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
options.maxiter=5000;
options.skip_error_calculation = 500;
method_name = "CD";# gradient descent
options.sketchsize = 100;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];
plot_outputs_Plots(OUTPUTS,savename)

rademacher-20-4
-------------------
It   | Error% | Residual |  Time   
-------------------
100  | 53.59  |  32.15  | 9.8752 
200  | 40.72  |  18.19  | 19.2877 
300  | 32.43  |  13.76  | 29.8606 
400  | 28.10  |  11.28  | 40.1743 
500  | 24.14  |  9.74  | 50.1443 
600  | 21.41  |  9.03  | 59.9515 
700  | 18.70  |  7.48  | 70.1964 
800  | 16.30  |  6.57  | 80.9024 
900  | 14.79  |  5.97  | 90.2410 
1000  | 13.84  |  5.52  | 100.5206 
1100  | 12.65  |  5.12  | 111.2109 
1200  | 11.32  |  4.75  | 124.0030 
1300  | 10.26  |  4.41  | 134.7009 
1400  | 9.48  |  4.12  | 144.6959 
1500  | 8.78  |  3.86  | 154.1839 
1600  | 8.19  |  3.61  | 163.3525 
1700  | 7.66  |  3.52  | 172.9010 
1800  | 7.64  |  3.28  | 183.0576 
1900  | 6.19  |  3.11  | 194.0066 
2000  | 6.48  |  2.90  | 203.9059 
2100  | 5.89  |  2.71  | 214.1605 
2200  | 5.55  |  2.52  | 224.1497 
2300  | 5.23  |  2.51  | 233.8348 
2400  | 4.98  |  2.34  | 243.3699 
2500  | 4.72  |  2.25  | 253.3765 
2600  | 4.49  |  2.14  | 263.1168 


In [133]:
plot_outputs_Plots(OUTPUTS,savename)

rcv1_train.multiclass


In [ ]:
options.maxiter=20000;
options.skip_error_calculation = 500;
method_name = "CD";# gradient descent
options.sketchsize = 100;#floor((prob.n)^(1/2));
@time output= solveRidge(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];


CD-100
-------------------
It   | Error% | Residual |  Time   
-------------------
500  | 9.33  |  5.29  | 45.1216 
